In [2]:
import torch
import stable_baselines3
import sys
import numpy as np

print("python version:", sys.version)
print("stable_baselines3 version:", stable_baselines3.__version__)
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("cuda version:", torch.version.cuda)
print("cudnn version:", torch.backends.cudnn.version())

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# set torch default device
torch.device(device)

c:\Users\tavar\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


python version: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
stable_baselines3 version: 2.1.0
torch version: 2.1.0+cu121
cuda available: True
cuda version: 12.1
cudnn version: 8801
device: cuda


device(type='cuda')

# Equation of Motion 3D Quadcopter

In [3]:
# reload sympy
from sympy import symbols, Matrix, lambdify, Array, sin, cos, tan

# Equations of motion 3D quadcopter from https://arxiv.org/pdf/2304.13460.pdf

# w1,w2,w3,w4 are the motor speeds normalized to [-1,1]
# u1,u2,u3,u4 are the motor commands normalized to [-1,1]

state = symbols('x y z v_x v_y v_z phi theta psi p q r w1 w2 w3 w4')
x,y,z,vx,vy,vz,phi,theta,psi,p,q,r,w1,w2,w3,w4 = state
control = symbols('u_1 u_2 u_3 u_4')
u1,u2,u3,u4 = control
disturbances = symbols('M_ext_x M_ext_y M_ext_z F_ext_x F_ext_y F_ext_z')
M_ext_x, M_ext_y, M_ext_z, F_ext_x, F_ext_y, F_ext_z = disturbances

# parameters from https://arxiv.org/pdf/2304.13460.pdf
g = 9.81
Ixx = 0.000906
Iyy = 0.001242
Izz = 0.002054

k_x  = 1.07933887e-05
k_y  = 9.65250793e-06
k_z  = 2.7862899e-05
k_w  = 4.36301076e-08
k_h  = 0.0625501332
k_p  = 1.4119331e-09
k_pv = -0.00797101848
k_q  = 1.21601884e-09
k_qv = 0.0129263739
k_r1 = 2.57035545e-06
k_r2 = 4.10923364e-07
k_rr = 0.000812932607

tau = 0.06
w_min = 3000
w_max = 11000


# Rotation matrix 
Rx = Matrix([[1, 0, 0], [0, cos(phi), -sin(phi)], [0, sin(phi), cos(phi)]])
Ry = Matrix([[cos(theta), 0, sin(theta)], [0, 1, 0], [-sin(theta), 0, cos(theta)]])
Rz = Matrix([[cos(psi), -sin(psi), 0], [sin(psi), cos(psi), 0], [0, 0, 1]])
R = Rz*Ry*Rx

# Body velocity
vbx, vby, vbz = R.T@Matrix([vx,vy,vz])

# normalized motor speeds to rpm
W1 = (w1+1)/2*(w_max-w_min) + w_min
W2 = (w2+1)/2*(w_max-w_min) + w_min
W3 = (w3+1)/2*(w_max-w_min) + w_min
W4 = (w4+1)/2*(w_max-w_min) + w_min

# first order delay
d_w1 = (u1 - w1)/tau
d_w2 = (u2 - w2)/tau
d_w3 = (u3 - w3)/tau

d_w4 = (u4 - w4)/tau

# derivative of rpm: d/dt[((w+1)/2*(w_max-w_min) + w_min)]
d_W1 = d_w1/2*(w_max-w_min)
d_W2 = d_w2/2*(w_max-w_min)
d_W3 = d_w3/2*(w_max-w_min)
d_W4 = d_w4/2*(w_max-w_min)

# Thrust and Drag
T = -k_w*(W1**2 + W2**2 + W3**2 + W4**2) - k_h*(vbx**2+vby**2) - k_z*vbz*(W1+W2+W3+W4) + F_ext_z
Dx = -k_x*vbx*(W1+W2+W3+W4) + F_ext_x
Dy = -k_y*vby*(W1+W2+W3+W4) + F_ext_y

# Moments
Mx = k_p*(W1**2-W2**2-W3**2+W4**2) + k_pv*vby + M_ext_x
My = k_q*(W1**2+W2**2-W3**2-W4**2) + k_qv*vbx + M_ext_y
Mz = k_r1*(-W1+W2-W3+W4) + k_r2*(-d_W1+d_W2-d_W3+d_W4) - k_rr*r + M_ext_z

# Dynamics
d_x = vx
d_y = vy
d_z = vz

d_vx, d_vy, d_vz = Matrix([0,0,g]) + R@Matrix([Dx, Dy,T])

d_phi   = p + q*sin(phi)*tan(theta) + r*cos(phi)*tan(theta)
d_theta = q*cos(phi) - r*sin(phi)
d_psi   = q*sin(phi)/cos(theta) + r*cos(phi)/cos(theta)

d_p     = (q*r*(Iyy-Izz) + Mx)/Ixx
d_q     = (p*r*(Izz-Ixx) + My)/Iyy
d_r     = (p*q*(Ixx-Iyy) + Mz)/Izz

# State space model
f = [d_x, d_y, d_z, d_vx, d_vy, d_vz, d_phi, d_theta, d_psi, d_p, d_q, d_r, d_w1, d_w2, d_w3, d_w4]

# lambdify
f_func = lambdify((Array(state), Array(control), Array(disturbances)), Array(f), 'numpy')

# extra functions
get_body_velocity = lambdify((Array(state),), Array([vbx, vby, vbz]), 'numpy')

In [4]:
# time f_func with random input
import numpy as np
state = np.random.rand(16)
control = np.random.rand(4)
disturbances = np.random.rand(6)
%timeit f_func(state, control, disturbances)

167 µs ± 3.73 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
# load thrust and moment model from NNDroneModel
thrust_model = torch.load('NNDroneModel/thrust_model.pt')
moment_model = torch.load('NNDroneModel/moment_model.pt')
print('thrust_model: ', thrust_model)
print('moment_model: ', moment_model)

# the models will be used for inference on the CPU
thrust_model.eval()
moment_model.eval()

# turn of grad
for p in thrust_model.parameters():
    p.requires_grad = False
for p in moment_model.parameters():
    p.requires_grad = False

# convert networks to numpy functions
thrust_model_np = lambda x: thrust_model(torch.from_numpy(x.astype(np.float32))).numpy()
moment_model_np = lambda x: moment_model(torch.from_numpy(x.astype(np.float32))).numpy()

# create a function that computes the thrust and moment from the world states (batched)
states = np.random.randn(4, 16).astype(np.float32)
states[0] = [0,1,2,3,4,5,0,0,0,9,10,11,12,13,14,15]
print('states: ', states)
vb = get_body_velocity(states.T).T
print('vb: ', vb)

def thrust_moment_model_world_states(states):
    w = states[:, 12:16]
    omega = states[:, 9:12]
    vb = get_body_velocity(states.T).T
    x_in_thrust = np.concatenate((w, vb), axis=1)
    # print('x_in_thrust: ', x_in_thrust)
    x_in_moment = np.concatenate((x_in_thrust, omega), axis=1)
    # print('x_in_moment: ', x_in_moment)
    return thrust_model_np(x_in_thrust), moment_model_np(x_in_moment)

# test the function
thrust, moment = thrust_moment_model_world_states(states)
print('thrust: ', thrust, thrust.dtype)
print('moment: ', moment, moment.dtype)

thrust_model:  Sequential(
  (0): Linear(in_features=7, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=1, bias=True)
)
moment_model:  Sequential(
  (0): Linear(in_features=10, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=3, bias=True)
)
states:  [[ 0.          1.          2.          3.          4.          5.
   0.          0.          0.          9.         10.         11.
  12.         13.         14.         15.        ]
 [ 1.1941702  -0.15103172 -0.02316403  0.23130536 -0.7297501  -0.39620686
   0.3027016   0.20915142  0.43229666 -1.1288526  -1.4191535  -1.8388615
   0.5741355   0.79878557  0.25183904  1.4648092 ]
 [-0.43822545  0.04664348  1.2009478  -0.9730012   0.59804755 -0.84937245
   1.2284176  -0.45284024  0.23149957  0.6168053  -1.9340041   2.1256685
  -1.729455   -0.7475703   0.18132651  1.5648422 ]
 [-2.4036968   0.7305871  -0.6377602   0.9977665  -0.32848334  0.43487218
  -1.4717647  -0.127951

# Define the 3D Quad Gates Environment

In [6]:
# Efficient vectorized version of the environment
from gymnasium.spaces.box import Box
from stable_baselines3.common.vec_env import VecEnv

class Quadcopter3DGates(VecEnv):
    def __init__(self,
                 num_envs,
                 gates_pos,
                 gate_yaw,
                 start_pos,
                 start_yaw,
                 gates_ahead=0,
                 pause_if_collision=False,
                 ):
        
        # Define the race track
        self.start_pos = start_pos.astype(np.float32)
        self.start_yaw = start_yaw.astype(np.float32)
        self.gate_pos = gates_pos.astype(np.float32)
        self.gate_yaw = gate_yaw.astype(np.float32)
        self.num_gates = gates_pos.shape[0]
        self.gates_ahead = gates_ahead
        
        # Pause if collision
        self.pause_if_collision = pause_if_collision

        # Calculate relative gates
        # pos,yaw of gate i in reference frame of gate i-1 (assumes a looped track)
        self.gate_pos_rel = np.zeros((self.num_gates,3), dtype=np.float32)
        self.gate_yaw_rel = np.zeros(self.num_gates, dtype=np.float32)
        for i in range(0,self.num_gates):
            self.gate_pos_rel[i] = self.gate_pos[i] - self.gate_pos[i-1]
            # Rotation matrix
            R = np.array([
                [np.cos(self.gate_yaw[i-1]), np.sin(self.gate_yaw[i-1])],
                [-np.sin(self.gate_yaw[i-1]), np.cos(self.gate_yaw[i-1])]
            ])
            self.gate_pos_rel[i,0:2] = R@self.gate_pos_rel[i,0:2]
            self.gate_yaw_rel[i] = self.gate_yaw[i] - self.gate_yaw[i-1]

        # Define the target gate for each environment
        self.target_gates = np.zeros(num_envs, dtype=int)

        # action space: [cmd1, cmd2, cmd3, cmd4]
        action_space = Box(low=-1, high=1, shape=(4,))

        # observation space: pos[G], vel[G], att[eulerB->G], rates[B], rpms, future_gates[G], future_gate_dirs[G]
        # [G] = reference frame aligned with target gate
        # [B] = body frame
        self.state_len = 16+4*self.gates_ahead + 4
        observation_space = Box(
            low  = np.array([-np.inf]*self.state_len),
            high = np.array([ np.inf]*self.state_len)
        )

        # Initialize the VecEnv
        VecEnv.__init__(self,num_envs, observation_space, action_space)

        # world state: pos[W], vel[W], att[eulerB->W], rates[B], rpms
        self.world_states = np.zeros((num_envs,16), dtype=np.float32)
        # observation state
        self.states = np.zeros((num_envs,self.state_len), dtype=np.float32)

        # Define any other environment-specific parameters
        self.max_steps = 4000       # Maximum number of steps in an episode
        self.dt = np.float32(0.005) # Time step duration

        self.step_counts = np.zeros(num_envs, dtype=int)
        self.actions = np.zeros((num_envs,4), dtype=np.float32)
        self.dones = np.zeros(num_envs, dtype=bool)
        self.final_gate_passed = np.zeros(num_envs, dtype=bool)

        self.update_states = self.update_states_gate
        
        self.disturbance_ranges = np.zeros((6,2), dtype=np.float32)
        self.disturbances = np.zeros((num_envs,6), dtype=np.float32)

        self.disturbance_scale = 1

        self.pause = False

    def update_states_world(self):
        self.states = self.world_states

    def update_states_gate(self):
        # Transform pos and vel in gate frame
        gate_pos = self.gate_pos[self.target_gates%self.num_gates]
        gate_yaw = self.gate_yaw[self.target_gates%self.num_gates]

        # Rotation matrix from world frame to gate frame
        R = np.array([
            [np.cos(gate_yaw), np.sin(gate_yaw)],
            [-np.sin(gate_yaw), np.cos(gate_yaw)]
        ]).transpose((2,1,0))

        # new state array to prevent the weird bug related to indexing ([:] syntax)
        new_states = np.zeros_like(self.states)

        # Update positions
        pos_W = self.world_states[:,0:3]
        pos_G = (pos_W[:,np.newaxis,0:2] - gate_pos[:,np.newaxis,0:2]) @ R
        new_states[:,0:2] = pos_G[:,0,:]
        new_states[:,2] = pos_W[:,2] - gate_pos[:,2]

        # Update velocities
        vel_W = self.world_states[:,3:6]
        vel_G = (vel_W[:,np.newaxis,0:2]) @ R
        new_states[:,3:5] = vel_G[:,0,:]
        new_states[:,5] = vel_W[:,2]

        # Update attitude
        new_states[:,6:8] = self.world_states[:,6:8]
        yaw = self.world_states[:,8] - gate_yaw
        yaw %= 2*np.pi
        yaw[yaw > np.pi] -= 2*np.pi
        yaw[yaw < -np.pi] += 2*np.pi
        new_states[:,8] = yaw

        # Update rates
        new_states[:,9:12] = self.world_states[:,9:12]

        # Update rpms
        new_states[:,12:16] = self.world_states[:,12:16]

        # Update future gates relative to current gate ([0,0,0,0] for out of bounds)
        for i in range(self.gates_ahead):
            indices = (self.target_gates+i+1)
            # loop when out of bounds
            indices = indices % self.num_gates
            valid = indices < self.num_gates
            new_states[valid,16+4*i:16+4*i+3] = self.gate_pos_rel[indices[valid]]
            new_states[valid,16+4*i+3] = self.gate_yaw_rel[indices[valid]]
        
        Mx = self.disturbances[:,0]
        My = self.disturbances[:,1]
        Mz = self.disturbances[:,2]
        Fz = self.disturbances[:,5]

        Mx_min = self.disturbance_ranges[0,0]
        Mx_max = self.disturbance_ranges[0,1]
        if Mx_min == Mx_max:
            Mx_min -= 1
            Mx_max += 1

        My_min = self.disturbance_ranges[1,0]
        My_max = self.disturbance_ranges[1,1]
        if My_min == My_max:
            My_min -= 1
            My_max += 1
        
        Mz_min = self.disturbance_ranges[2,0]
        Mz_max = self.disturbance_ranges[2,1]
        if Mz_min == Mz_max:
            Mz_min -= 1
            Mz_max += 1

        Fz_min = self.disturbance_ranges[5,0]
        Fz_max = self.disturbance_ranges[5,1]
        if Fz_min == Fz_max:
            Fz_min -= 1
            Fz_max += 1

        new_states[:,16+4*self.gates_ahead:] = np.array([
            2*(Mx-Mx_min)/(Mx_max-Mx_min)-1, # Mx
            2*(My-My_min)/(My_max-My_min)-1, # My
            2*(Mz-Mz_min)/(Mz_max-Mz_min)-1, # Mz
            2*(Fz-Fz_min)/(Fz_max-Fz_min)-1, # Fz
        ]).T

        self.states = new_states

    def reset_(self, dones):
        num_reset = dones.sum()

        x0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[0]
        y0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[1]
        z0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[2]
        
        vx0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        vy0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        vz0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        
        phi0   = np.random.uniform(-np.pi/9,np.pi/9, size=(num_reset,))
        theta0 = np.random.uniform(-np.pi/9,np.pi/9, size=(num_reset,)) + self.start_yaw
        psi0   = np.random.uniform(-np.pi,np.pi, size=(num_reset,))
        
        p0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        q0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        r0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        
        w10 = np.random.uniform(-1,1, size=(num_reset,))
        w20 = np.random.uniform(-1,1, size=(num_reset,))
        w30 = np.random.uniform(-1,1, size=(num_reset,))
        w40 = np.random.uniform(-1,1, size=(num_reset,))

        self.world_states[dones] = np.stack([x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0, p0, q0, r0, w10, w20, w30, w40], axis=1)

        self.step_counts[dones] = np.zeros(num_reset)
        
        self.target_gates[dones] = np.zeros(num_reset, dtype=int)

        M_ext_x = np.random.uniform(self.disturbance_ranges[0,0], self.disturbance_ranges[0,1], size=(num_reset,))
        M_ext_y = np.random.uniform(self.disturbance_ranges[1,0], self.disturbance_ranges[1,1], size=(num_reset,))
        M_ext_z = np.random.uniform(self.disturbance_ranges[2,0], self.disturbance_ranges[2,1], size=(num_reset,))
        F_ext_x = np.random.uniform(self.disturbance_ranges[3,0], self.disturbance_ranges[3,1], size=(num_reset,))
        F_ext_y = np.random.uniform(self.disturbance_ranges[4,0], self.disturbance_ranges[4,1], size=(num_reset,))
        F_ext_z = np.random.uniform(self.disturbance_ranges[5,0], self.disturbance_ranges[5,1], size=(num_reset,))
        
        self.disturbances[dones] = self.disturbance_scale*np.stack([M_ext_x, M_ext_y, M_ext_z, F_ext_x, F_ext_y, F_ext_z], axis=1)

        # update states
        self.update_states()
        return self.states
    
    def reset(self):
        return self.reset_(np.ones(self.num_envs, dtype=bool))

    def step_async(self, actions):
        self.actions = actions
    
    def step_wait(self):
        # Residual nn model
        d = np.zeros((self.num_envs,6), dtype=np.float32)
        thurst, moment = thrust_moment_model_world_states(self.world_states)
        d[:,0:3] = moment
        d[:,5:6] = thurst

        # Add disturbances
        d += self.disturbances
        

        new_states = self.world_states + self.dt*f_func(self.world_states.T, self.actions.T, d.T).T
        # new_states = self.world_states + self.dt*f_func(self.world_states.T, self.actions.T, self.disturbances[self.step_counts].T).T
        self.step_counts += 1

        pos_old = self.world_states[:,0:3]
        pos_new = new_states[:,0:3]
        pos_gate = self.gate_pos[self.target_gates%self.num_gates]
        yaw_gate = self.gate_yaw[self.target_gates%self.num_gates]

        # Rewards
        d2g_old = np.linalg.norm(pos_old - pos_gate, axis=1)
        d2g_new = np.linalg.norm(pos_new - pos_gate, axis=1)
        rat_penalty = 0*0.01*np.linalg.norm(new_states[:,9:12], axis=1)
        rewards = d2g_old - d2g_new - rat_penalty
        
        # Gate passing/collision
        normal = np.array([np.cos(yaw_gate), np.sin(yaw_gate)]).T
        # dot product of normal and position vector over axis 1
        pos_old_projected = (pos_old[:,0]-pos_gate[:,0])*normal[:,0] + (pos_old[:,1]-pos_gate[:,1])*normal[:,1]
        pos_new_projected = (pos_new[:,0]-pos_gate[:,0])*normal[:,0] + (pos_new[:,1]-pos_gate[:,1])*normal[:,1]
        passed_gate_plane = (pos_old_projected < 0) & (pos_new_projected > 0)
        gate_passed = passed_gate_plane & np.all(np.abs(pos_new - pos_gate)<0.5, axis=1)
        gate_collision = passed_gate_plane & np.any(np.abs(pos_new - pos_gate)>0.5, axis=1)

        # Gate reward + dist penalty
        rewards[gate_passed] = 10 - 10*d2g_new[gate_passed]
        
        # Gate collision penalty
        rewards[gate_collision] = -10

        # Ground collision penalty (z > 0)
        ground_collision = new_states[:,2] > 0
        rewards[ground_collision] = -10
        
        # Check out of bounds
        # outside grid abs(x,y)>10
        # prevent numerical issues: abs(p,q,r) < 1000
        out_of_bounds = np.any(np.abs(new_states[:,0:2]) > 10, axis=1) | np.any(np.abs(new_states[:,9:12]) > 1000, axis=1)
        rewards[out_of_bounds] = -10
        
        # Check number of steps
        max_steps_reached = self.step_counts >= self.max_steps

        # Update target gate
        self.target_gates[gate_passed] += 1
        self.target_gates[gate_passed] %= self.num_gates
        
        # Check if final gate has been passed
        # self.final_gate_passed = self.target_gates >= self.num_gates

        # give reward for passing final gate
        rewards[self.final_gate_passed] = 10
        
        # Check if the episode is done
        dones = max_steps_reached | ground_collision | gate_collision | out_of_bounds #| self.final_gate_passed
        self.dones = dones
        
        # Pause if collision
        if self.pause:
            dones = dones & ~dones
            self.dones = dones
        elif self.pause_if_collision:
            # dones = max_steps_reached | final_gate_passed | out_of_bounds
            update = ~dones #~(gate_collision | ground_collision)
            # Update world states
            self.world_states[update] = new_states[update]
            self.update_states()
            # Reset env if done (and update states)
            # self.reset_(dones)
        else:
            # Update world states
            self.world_states = new_states
            # reset env if done (and update states)
            self.reset_(dones)


        # Write info dicts
        infos = [{}] * self.num_envs
        for i in range(self.num_envs):
            if dones[i]:
                infos[i]["terminal_observation"] = self.states[i]
            if max_steps_reached[i]:
                infos[i]["TimeLimit.truncated"] = True
        return self.states, rewards, dones, infos
    
    def close(self):
        pass

    def seed(self, seed=None):
        pass

    def get_attr(self, attr_name, indices=None):
        pass

    def set_attr(self, attr_name, value, indices=None):
        pass

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        pass

    def env_is_wrapped(self, wrapper_class, indices=None):
        return [False]*self.num_envs

    def render(self, mode='human'):
        # Outputs a dict containing all information for rendering
        state_dict = dict(zip(['x','y','z','vx','vy','vz','phi','theta','psi','p','q','r','w1','w2','w3','w4'], self.world_states.T))
        # Rescale actions to [0,1] for rendering
        action_dict = dict(zip(['u1','u2','u3','u4'], (np.array(self.actions.T)+1)/2))
        return {**state_dict, **action_dict}

# Define Race Track

In [7]:
import importlib
from quadcopter_animation import animation
importlib.reload(animation)

# Define the race track
# gate_pos = np.array([
#     [-1.5,-2,-1.5],
#     [1.5,2,-1.5],
#     [1.5,-2,-2.5],
#     [-1.5,2,-1.5]
# ]*2)

gate_pos = np.array([
    [ 2,-1.5,-1.5],
    [ 2, 1.5,-1.5],
    [-2, 1.5,-1.5],
    [-2,-1.5,-1.5]
]*2)

# gate_pos = np.array([
#     [ 2, -2.5, -1],
#     [ 3, 0, -1.5],
#     [ 0.5, 0, -1.5],
#     [-2, -1.5, -2],
#     [-2.5, 1, -2],
#     [ 0, 1.5,-1.5],
#     [ 2, 2, -2],
#     [ 3, 0, -1.5],
#     [-1, 0, -1.5],
#     [-3.5, 0, -1.5],
#     [-1.5, -3.5,-1.5],
#     [ 1, -1.5, -1],
#     [3.5,-3,-1.5]
# ]*2)

# gate_yaw = np.array([
#     0,
#     0,
#     np.pi,
#     np.pi
# ]*2)
gate_yaw = np.array([
    np.pi/4,
    3*np.pi/4,
    5*np.pi/4,
    7*np.pi/4
]*2)

# gate_yaw = np.array([
#     np.pi / 2,
#     np.pi / 2,
#     3 * np.pi / 2,
#     3 * np.pi / 4, 
#     np.pi / 4,
#     0,
#     0,
#     3 * np.pi / 2,
#     np.pi,
#     3 * np.pi / 2,
#     0,
#     np.pi / 2,
#     3 * np.pi / 2,
# ]*2)

# start_pos = gate_pos[0] - np.array([2,0,0])
start_pos = gate_pos[0] #- np.array([2,0,0]
start_yaw = gate_yaw[0] #- np.array([2,0,0])

num = 10
env = Quadcopter3DGates(num_envs=num, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos,start_yaw=start_yaw, pause_if_collision=False)

# Run a random agent
env.reset()

done = False
def run():
    global done
    action = np.random.uniform(-1,1, size=(num,4))
    state, reward, done, _ = env.step(action)
    # print(state[0][-4:])
    # print(env.disturbances[0])
    if reward[0] > 1:
        print("reward:", reward)
    return env.render()

animation.view(run, gate_pos=gate_pos, gate_yaw=gate_yaw) #, record_steps=1000, show_window=True)

# Train PPO Model

In [8]:
import os
from stable_baselines3 import PPO
from datetime import datetime
from stable_baselines3.common.vec_env import VecMonitor
import importlib
from quadcopter_animation import animation

models_dir = 'models/E2E'
log_dir = 'logs/E2E'
video_log_dir = 'videos/E2E'

if not os.path.exists(models_dir):
    os.makedirs(models_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(video_log_dir):
    os.makedirs(video_log_dir)

# Date and time string for unique folder names
datetime_str = datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the environment
env = Quadcopter3DGates(num_envs=100, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, start_yaw=start_yaw, gates_ahead=1)
test_env = Quadcopter3DGates(num_envs=10, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=start_pos, start_yaw=start_yaw, gates_ahead=1, pause_if_collision=True)

# Wrap the environment in a Monitor wrapper
env = VecMonitor(env)

# disturbance ranges
disturbance_ranges = np.array([
    [-0.03, 0.03],
    [-0.03, 0.03],
    [-0.01, 0.01],
    [0, 0],
    [0, 0],
    [-0.5, 0.5],
])
env.venv.disturbance_ranges = disturbance_ranges
test_env.disturbance_ranges = disturbance_ranges

# MODEL DEFINITION
policy_kwargs = dict(activation_fn=torch.nn.ReLU, net_arch=[dict(pi=[120,120,120], vf=[120,120,120])], log_std_init = 0)
model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=policy_kwargs,
    verbose=0,
    tensorboard_log=log_dir,
    n_steps=1000,
    batch_size=5000,
    n_epochs=10,
    gamma=0.999
)

print(model.policy)
print(model.num_timesteps)

def animate_policy(model, env, deterministic=False, log_times=False, **kwargs):
    env.reset()
    def run():
        actions, _ = model.predict(env.states, deterministic=deterministic)

        states, rewards, dones, infos = env.step(actions)
        if log_times:
            if rewards[0] == 10:
                print(env.step_counts[0]*env.dt)
        return env.render()
    animation.view(run, gate_pos=env.gate_pos, gate_yaw=env.gate_yaw, **kwargs)

# animate untrained policy (use this to set the recording camera position)
animate_policy(model, test_env)

# training loop saves model every 10 policy rollouts and saves a video animation
def train(model, test_env, TIMESTEPS, log_name):
        model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=log_name)
        time_steps = model.num_timesteps
        # save model
        model.save(models_dir + '/' + log_name + '/' + str(time_steps))
        # save policy animation
        animate_policy(
            model,
            test_env,
            record_steps=2000,
            record_file=video_log_dir + '/' + log_name + '/' + str(time_steps) + '.mp4',
            show_window=False
        )

c:\Users\tavar\anaconda3\lib\site-packages\stable_baselines3\common\policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


ActorCriticPolicy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=24, out_features=120, bias=True)
      (1): ReLU()
      (2): Linear(in_features=120, out_features=120, bias=True)
      (3): ReLU()
      (4): Linear(in_features=120, out_features=120, bias=True)
      (5): ReLU()
    )
    (value_net): Sequential(
      (0): Linear(in_features=24, out_features=120, bias=True)
      (1): ReLU()
      (2): Linear(in_features=120, out_features=120, bias=True)
      (3): ReLU()
      (4): Linear(in_features=120, out_features=120, bias=True)
      (5): ReLU()
    )
  )
  (action_net): Linear(in_features=120, out_features=4, bias=True)
  (value_net): Linear(in_fea

In [25]:
# run training loop
import random
test_env = Quadcopter3DGates(num_envs=10, gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=gate_pos[0], start_yaw=gate_yaw[0], gates_ahead=1, pause_if_collision=True)
for _ in range(0, 10000000000):
    # save every 10 policy rollouts
    TIMESTEPS = model.n_steps*env.num_envs*30
    # start_gate = random.randint(0, int(len(gate_yaw) / 2))
    train(model, test_env, TIMESTEPS, 'test4')

recording started
recording ended
recording saved in videos/E2E/test4/3000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/6000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/9000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/12000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/15000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/18000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/21000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/24000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/27000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/30000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/33000000.mp4
recording started
recording ended
recording saved in videos/E2E/test4/36000000.mp4
recordi

KeyboardInterrupt: 

# Simulate PPO model

In [8]:
animate_policy(model, test_env)

# Generate dataset with the model

In [9]:
import torch.nn as nn

# E2E_Net
path = 'models/E2E/test4/57000000.zip'

model = PPO.load(path)

# get network
network = list(model.policy.mlp_extractor.policy_net) + [model.policy.action_net]
network = nn.Sequential(*network)
print('NETWORK:')
print(network)

print(model.policy.action_dist)
print(model.policy.log_std)
print(model.policy.log_std.exp())
network_std = model.policy.log_std.exp().cpu().detach().numpy()
print(network_std)



NETWORK:
Sequential(
  (0): Linear(in_features=24, out_features=120, bias=True)
  (1): ReLU()
  (2): Linear(in_features=120, out_features=120, bias=True)
  (3): ReLU()
  (4): Linear(in_features=120, out_features=120, bias=True)
  (5): ReLU()
  (6): Linear(in_features=120, out_features=4, bias=True)
)
Parameter containing:
tensor([-0.6471, -0.8764, -0.7122, -0.9698], device='cuda:0',
       requires_grad=True)
tensor([0.5236, 0.4163, 0.4905, 0.3792], device='cuda:0',
       grad_fn=<ExpBackward0>)
[0.52355987 0.4162943  0.4905478  0.37916064]


In [14]:
class Quadcopter3DGatesSim(VecEnv):
    def __init__(self,
                 num_envs,
                 gates_pos,
                 gate_yaw,
                 start_pos,
                 gates_ahead=0,
                 pause_if_collision=False,
                 start_gate=0
                 ):
        
        # Define the race track
        self.start_pos = start_pos.astype(np.float32)
        self.gate_pos = gates_pos.astype(np.float32)
        self.gate_yaw = gate_yaw.astype(np.float32)
        self.num_gates = gates_pos.shape[0]
        self.gates_ahead = gates_ahead
        self.start_gate=start_gate
        
        # Pause if collision
        self.pause_if_collision = pause_if_collision

        # Calculate relative gates
        # pos,yaw of gate i in reference frame of gate i-1 (assumes a looped track)
        self.gate_pos_rel = np.zeros((self.num_gates,3), dtype=np.float32)
        self.gate_yaw_rel = np.zeros(self.num_gates, dtype=np.float32)
        for i in range(0,self.num_gates):
            self.gate_pos_rel[i] = self.gate_pos[i] - self.gate_pos[i-1]
            # Rotation matrix
            R = np.array([
                [np.cos(self.gate_yaw[i-1]), np.sin(self.gate_yaw[i-1])],
                [-np.sin(self.gate_yaw[i-1]), np.cos(self.gate_yaw[i-1])]
            ])
            self.gate_pos_rel[i,0:2] = R@self.gate_pos_rel[i,0:2]
            self.gate_yaw_rel[i] = self.gate_yaw[i] - self.gate_yaw[i-1]

        # Define the target gate for each environment
        self.target_gates = np.zeros(num_envs, dtype=int)

        # action space: [cmd1, cmd2, cmd3, cmd4]
        action_space = Box(low=-1, high=1, shape=(4,))

        # observation space: pos[G], vel[G], att[eulerB->G], rates[B], rpms, future_gates[G], future_gate_dirs[G]
        # [G] = reference frame aligned with target gate
        # [B] = body frame
        self.state_len = 16+4*self.gates_ahead + 4
        observation_space = Box(
            low  = np.array([-np.inf]*self.state_len),
            high = np.array([ np.inf]*self.state_len)
        )

        # Initialize the VecEnv
        VecEnv.__init__(self,num_envs, observation_space, action_space)

        # world state: pos[W], vel[W], att[eulerB->W], rates[B], rpms
        self.world_states = np.zeros((num_envs,16), dtype=np.float32)
        # observation state
        self.states = np.zeros((num_envs,self.state_len), dtype=np.float32)

        # Define any other environment-specific parameters
        self.max_steps = 20000      # Maximum number of steps in an episode
        self.dt = np.float32(0.005) # Time step duration

        self.step_counts = np.zeros(num_envs, dtype=int)
        self.actions = np.zeros((num_envs,4), dtype=np.float32)
        self.dones = np.zeros(num_envs, dtype=bool)
        self.final_gate_passed = np.zeros(num_envs, dtype=bool)

        self.update_states = self.update_states_gate
        
        self.disturbance_ranges = np.zeros((6,2), dtype=np.float32)
        self.disturbances = np.zeros((num_envs,6), dtype=np.float32)

        self.disturbance_scale = 1

        self.pause = False

    def update_states_world(self):
        self.states = self.world_states

    def update_states_gate(self):
        # Transform pos and vel in gate frame
        gate_pos = self.gate_pos[self.target_gates%self.num_gates]
        gate_yaw = self.gate_yaw[self.target_gates%self.num_gates]

        # Rotation matrix from world frame to gate frame
        R = np.array([
            [np.cos(gate_yaw), np.sin(gate_yaw)],
            [-np.sin(gate_yaw), np.cos(gate_yaw)]
        ]).transpose((2,1,0))

        # new state array to prevent the weird bug related to indexing ([:] syntax)
        new_states = np.zeros_like(self.states)

        # Update positions
        pos_W = self.world_states[:,0:3]
        pos_G = (pos_W[:,np.newaxis,0:2] - gate_pos[:,np.newaxis,0:2]) @ R
        new_states[:,0:2] = pos_G[:,0,:]
        new_states[:,2] = pos_W[:,2] - gate_pos[:,2]

        # Update velocities
        vel_W = self.world_states[:,3:6]
        vel_G = (vel_W[:,np.newaxis,0:2]) @ R
        new_states[:,3:5] = vel_G[:,0,:]
        new_states[:,5] = vel_W[:,2]

        # Update attitude
        new_states[:,6:8] = self.world_states[:,6:8]
        yaw = self.world_states[:,8] - gate_yaw
        yaw %= 2*np.pi
        yaw[yaw > np.pi] -= 2*np.pi
        yaw[yaw < -np.pi] += 2*np.pi
        new_states[:,8] = yaw

        # Update rates
        new_states[:,9:12] = self.world_states[:,9:12]

        # Update rpms
        new_states[:,12:16] = self.world_states[:,12:16]

        # Update future gates relative to current gate ([0,0,0,0] for out of bounds)
        for i in range(self.gates_ahead):
            indices = (self.target_gates+i+1)
            # loop when out of bounds
            indices = indices % self.num_gates
            valid = indices < self.num_gates
            new_states[valid,16+4*i:16+4*i+3] = self.gate_pos_rel[indices[valid]]
            new_states[valid,16+4*i+3] = self.gate_yaw_rel[indices[valid]]
        
        Mx = self.disturbances[:,0]
        My = self.disturbances[:,1]
        Mz = self.disturbances[:,2]
        Fz = self.disturbances[:,5]

        Mx_min = self.disturbance_ranges[0,0]
        Mx_max = self.disturbance_ranges[0,1]
        if Mx_min == Mx_max:
            Mx_min -= 1
            Mx_max += 1

        My_min = self.disturbance_ranges[1,0]
        My_max = self.disturbance_ranges[1,1]
        if My_min == My_max:
            My_min -= 1
            My_max += 1
        
        Mz_min = self.disturbance_ranges[2,0]
        Mz_max = self.disturbance_ranges[2,1]
        if Mz_min == Mz_max:
            Mz_min -= 1
            Mz_max += 1

        Fz_min = self.disturbance_ranges[5,0]
        Fz_max = self.disturbance_ranges[5,1]
        if Fz_min == Fz_max:
            Fz_min -= 1
            Fz_max += 1

        new_states[:,16+4*self.gates_ahead:] = np.array([
            2*(Mx-Mx_min)/(Mx_max-Mx_min)-1, # Mx
            2*(My-My_min)/(My_max-My_min)-1, # My
            2*(Mz-Mz_min)/(Mz_max-Mz_min)-1, # Mz
            2*(Fz-Fz_min)/(Fz_max-Fz_min)-1, # Fz
        ]).T

        self.states = new_states

    def reset_(self, dones):
        num_reset = dones.sum()

        x0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[0]
        y0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[1]
        z0 = np.random.uniform(-0.5,0.5, size=(num_reset,)) + self.start_pos[2]
        
        vx0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        vy0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        vz0 = np.random.uniform(-0.5,0.5, size=(num_reset,))
        
        phi0   = np.random.uniform(-np.pi/9,np.pi/9, size=(num_reset,))
        theta0 = np.random.uniform(-np.pi/9,np.pi/9, size=(num_reset,))
        psi0   = np.random.uniform(-np.pi,np.pi, size=(num_reset,))
        
        p0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        q0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        r0 = np.random.uniform(-0.1,0.1, size=(num_reset,))
        
        w10 = np.random.uniform(-1,1, size=(num_reset,))
        w20 = np.random.uniform(-1,1, size=(num_reset,))
        w30 = np.random.uniform(-1,1, size=(num_reset,))
        w40 = np.random.uniform(-1,1, size=(num_reset,))

        self.world_states[dones] = np.stack([x0, y0, z0, vx0, vy0, vz0, phi0, theta0, psi0, p0, q0, r0, w10, w20, w30, w40], axis=1)

        self.step_counts[dones] = np.zeros(num_reset)
        
        self.target_gates[dones] = np.zeros(num_reset, dtype=int)

        M_ext_x = np.random.uniform(self.disturbance_ranges[0,0], self.disturbance_ranges[0,1], size=(num_reset,))
        M_ext_y = np.random.uniform(self.disturbance_ranges[1,0], self.disturbance_ranges[1,1], size=(num_reset,))
        M_ext_z = np.random.uniform(self.disturbance_ranges[2,0], self.disturbance_ranges[2,1], size=(num_reset,))
        F_ext_x = np.random.uniform(self.disturbance_ranges[3,0], self.disturbance_ranges[3,1], size=(num_reset,))
        F_ext_y = np.random.uniform(self.disturbance_ranges[4,0], self.disturbance_ranges[4,1], size=(num_reset,))
        F_ext_z = np.random.uniform(self.disturbance_ranges[5,0], self.disturbance_ranges[5,1], size=(num_reset,))
        
        self.disturbances[dones] = self.disturbance_scale*np.stack([M_ext_x, M_ext_y, M_ext_z, F_ext_x, F_ext_y, F_ext_z], axis=1)

        # update states
        self.update_states()
        return self.states
    
    def reset(self):
        return self.reset_(np.ones(self.num_envs, dtype=bool))

    def step_async(self, actions):
        self.actions = actions
    
    def step_wait(self):
        # Residual nn model
        d = np.zeros((self.num_envs,6), dtype=np.float32)
        thurst, moment = thrust_moment_model_world_states(self.world_states)
        d[:,0:3] = moment
        d[:,5:6] = thurst

        # Add disturbances
        d += self.disturbances
        

        new_states = self.world_states + self.dt*f_func(self.world_states.T, self.actions.T, d.T).T
        # new_states = self.world_states + self.dt*f_func(self.world_states.T, self.actions.T, self.disturbances[self.step_counts].T).T
        self.step_counts += 1

        pos_old = self.world_states[:,0:3]
        pos_new = new_states[:,0:3]
        pos_gate = self.gate_pos[self.target_gates%self.num_gates]
        yaw_gate = self.gate_yaw[self.target_gates%self.num_gates]

        # Rewards
        d2g_old = np.linalg.norm(pos_old - pos_gate, axis=1)
        d2g_new = np.linalg.norm(pos_new - pos_gate, axis=1)
        rat_penalty = 0*0.01*np.linalg.norm(new_states[:,9:12], axis=1)
        rewards = d2g_old - d2g_new - rat_penalty
        
        # Gate passing/collision
        normal = np.array([np.cos(yaw_gate), np.sin(yaw_gate)]).T
        # dot product of normal and position vector over axis 1
        pos_old_projected = (pos_old[:,0]-pos_gate[:,0])*normal[:,0] + (pos_old[:,1]-pos_gate[:,1])*normal[:,1]
        pos_new_projected = (pos_new[:,0]-pos_gate[:,0])*normal[:,0] + (pos_new[:,1]-pos_gate[:,1])*normal[:,1]
        passed_gate_plane = (pos_old_projected < 0) & (pos_new_projected > 0)
        gate_passed = passed_gate_plane & np.all(np.abs(pos_new - pos_gate)<0.5, axis=1)
        gate_collision = passed_gate_plane & np.any(np.abs(pos_new - pos_gate)>0.5, axis=1)
        
        # Gate reward + dist penalty
        rewards[gate_passed] = 10 - 10*d2g_new[gate_passed]
        
        # Gate collision penalty
        rewards[gate_collision] = -10

        # Ground collision penalty (z > 0)
        ground_collision = new_states[:,2] > 0
        rewards[ground_collision] = -10
        
        # Check out of bounds
        # outside grid abs(x,y)>10
        # prevent numerical issues: abs(p,q,r) < 1000
        out_of_bounds = np.any(np.abs(new_states[:,0:2]) > 10, axis=1) | np.any(np.abs(new_states[:,9:12]) > 1000, axis=1)
        rewards[out_of_bounds] = -10
        
        # Check number of steps
        max_steps_reached = self.step_counts >= self.max_steps

        # Update target gate
        self.target_gates[gate_passed] += 1
        self.target_gates[gate_passed] %= self.num_gates
        
        # Check if final gate has been passed
        # self.final_gate_passed = self.target_gates >= self.num_gates

        # give reward for passing final gate
        rewards[self.final_gate_passed] = 10
        
        # Check if the episode is done
        dones = max_steps_reached | ground_collision | gate_collision | out_of_bounds #| self.final_gate_passed
        self.dones = dones
        
        # Pause if collision
        if self.pause:
            dones = dones & ~dones
            self.dones = dones
        elif self.pause_if_collision:
            # dones = max_steps_reached | final_gate_passed | out_of_bounds
            update = ~dones #~(gate_collision | ground_collision)
            # Update world states
            self.world_states[update] = new_states[update]
            self.update_states()
            # Reset env if done (and update states)
            # self.reset_(dones)
        else:
            # Update world states
            self.world_states = new_states
            # reset env if done (and update states)
            self.reset_(dones)


        # Write info dicts
        infos = [{}] * self.num_envs
        for i in range(self.num_envs):
            if dones[i]:
                infos[i]["terminal_observation"] = self.states[i]
            if max_steps_reached[i]:
                infos[i]["TimeLimit.truncated"] = True
        return self.states, rewards, dones, infos
    
    def close(self):
        pass

    def seed(self, seed=None):
        pass

    def get_attr(self, attr_name, indices=None):
        pass

    def set_attr(self, attr_name, value, indices=None):
        pass

    def env_method(self, method_name, *method_args, indices=None, **method_kwargs):
        pass

    def env_is_wrapped(self, wrapper_class, indices=None):
        return [False]*self.num_envs

    def render(self, mode='human'):
        # Outputs a dict containing all information for rendering
        state_dict = dict(zip(['x','y','z','vx','vy','vz','phi','theta','psi','p','q','r','w1','w2','w3','w4'], self.world_states.T))
        # Rescale actions to [0,1] for rendering
        action_dict = dict(zip(['u1','u2','u3','u4'], (np.array(self.actions.T)+1)/2))
        disturbances_dict = dict(zip(['Mx','My','Mz','Fz'], (np.array([self.disturbances[0,0]]).astype(np.float32),
                                                             np.array([self.disturbances[0,1]]).astype(np.float32),
                                                             np.array([self.disturbances[0,2]]).astype(np.float32),
                                                             np.array([self.disturbances[0,5]]).astype(np.float32))))
        # print([self.gate_pos[1]].append(self.gate_yaw[1]))

        next_gate_values = self.gate_pos[(self.target_gates[0] + self.start_gate + 1) % self.num_gates].tolist()
        next_gate_values.append(self.gate_yaw[(self.target_gates[0] + self.start_gate + 2) % self.num_gates])
        next_gate_dict = dict(zip(['gate_x','gate_y','gate_z','gate_yaw'], (np.array([next_gate_values[0]]).astype(np.float32),
                                                                            np.array([next_gate_values[1]]).astype(np.float32),
                                                                            np.array([next_gate_values[2]]).astype(np.float32),
                                                                            np.array([next_gate_values[3]]).astype(np.float32))))
        return {**state_dict, **disturbances_dict, **next_gate_dict, **action_dict}

In [15]:
import csv
import time

def make_dataset(model, env, round, deterministic=False, log_times=False, **kwargs):
    env.reset()
    initial_time = time.time()
    def run():
        actions, _ = model.predict(env.states, deterministic=deterministic)
        states, rewards, dones, infos = env.step(actions)
        if log_times:
            if rewards[0] == 10:
                print(env.step_counts[0]*env.dt)
        return env.render(), dones
    
    csv_file_path = 'data_control_test_long_run.csv'
    file_exists = True
    try:
        with open(csv_file_path, 'r') as file:
            pass
    except FileNotFoundError:
        file_exists = False
    
    collision = False

    while (not collision):
        existing_data = []
        env.reset()
        initial_time = time.time()
        while (len(existing_data) < 10000):
            data, dones = run()
            if dones[0]:
                print(len(existing_data))
                break
            new_data = {key: value[0] for key, value in data.items()}

            # Add your dictionary to the existing data
            existing_data.append(new_data)

        # Write the updated data to the CSV file
        if (not dones[0]):
            collision=True
            with open(csv_file_path, 'a', newline='') as file:
                fieldnames = existing_data[0].keys()
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                # Write header only if the file is newly created
                if not file_exists:
                    writer.writeheader()
                    file_exists = True

                # Write new values
                for new_data in existing_data:
                    writer.writerow(new_data)


In [16]:
import random
for i in range(1):
    print("Start:", i)
    start_gate = random.choice([0])
    print("Start_gate", start_gate)
    test_env = Quadcopter3DGatesSim(num_envs=1,gates_pos=gate_pos, gate_yaw=gate_yaw, start_pos=gate_pos[start_gate], gates_ahead=1, pause_if_collision=True, start_gate=start_gate)
    ranges = np.array([
        [-0.03, 0.03],
        [-0.03, 0.03],
        [-0.01, 0.01],
        [0, 0],
        [0, 0],
        [-0.5, 0.5],
    ])
    test_env.disturbance_ranges = ranges

    make_dataset(model, test_env, i, deterministic=False, log_times=True)

Start: 0
Start_gate 0
